In [68]:
import re
lines = []
Questions = []
Golds = []
QSplitted = []
GSplitted = []
col = []
lines = list(open("spider_multi_col_22000.txt", "r", encoding='utf-8').readlines())
# you may also want to remove whitespace characters like `\n` at the end of each line
for i in range(0, len(lines)):
    if i%3 == 0:
        Questions.append(lines[i])
        QSplitted.append(re.split(', and |, | and |and |\n',lines[i].strip("\n")))
    if i%3 == 1:
        Golds.append(lines[i])
        column = lines[i][7: lines[i].index("FROM")]
        col.append(re.split(' ,  ',column.strip(' '))) 

special = {"avg": "average ", "min": "minimum ", "max": "maximum ", 
         "sum": "sum of ", "count": "number of ", "Count": "number of ", "COUNT": "number of ", "count(DISTINCT": "number of different "}

for c in col:
    for i in range(0, len(c)):
        if "(" in c[i]:
            if 'count(DISTINCT' in c[i]:
                op = 'count(DISTINCT'
                att = c[i].strip('count(DISTINCT').strip(' ').strip(')')
            else:
                op = c[i][0:c[i].index('(')]
                att = c[i][c[i].index('(')+1:c[i].index(')')]
            if att == "*":
                c[i] = special[op.strip(' ')]
            else:
                c[i] = special[op.strip(' ')] + att
print(QSplitted)
print(len(col))
same = []

for i in range(0,len(col)):
    same.append(len(QSplitted[i]) < len(col[i]))

print(same)
                
#testQ = []
#testG = []
#testQS = []
#testGS = []
#for i in range(0, 10):
    #print(i)
    #testQ.append(Questions[i])
    #testG.append(Golds[i])
    #testQS.append(QSplitted[i])
    #testGS.append(GSplitted[i])
    #print(Questions[i])
    #print(QSplitted[i])
    #print(Golds[i])
    #print(GSplitted[i])


[['Show name', 'country', 'age for all singers ordered by age from the oldest to the youngest.'], ['What are the names', 'countries', 'ages for every singer in descending order of age?'], ['What is the average', 'minimum', 'maximum age of all singers from France?'], ['What is the average', 'minimum', 'maximum age for all French singers?'], ['Show the name', 'the release year of the song by the youngest singer.'], ['What are the names', 'release years for all the songs of the youngest singer?'], ['Show all countries', 'the number of singers in each country.'], ['How many singers are from each country?'], ['Show location', 'name for all stadiums with a capacity between 5000', '10000.'], ['What are the locations', 'names of all stations with capacity between 5000', '10000?'], ['What is the average', 'the maximum capacity of all stadiums?'], ['What is the average', 'maximum capacities for all stations?'], ['What is the name', 'capacity for the stadium with highest average attendance?'], ['